# We've Got You Covered - Backup Management with dbatools
## Abstract
Every database administrator needs to deal with backups - scheduled, ad-hoc, restoring, testing, and managing them. dbatools provides a rich set of functionality for managing all of this, without having to write a lot of T-SQL or dig through layers upon layers of dialog boxes in Manaagement Studio.
## About Me
| At Work |At Home|
|---------|-------|
|Production DBA|Thing 1 |
|Development DBA|Thing 2 |
|Web Developer|Thing 3 |
|Web Server Administrator|Thing 4 |
|Systems Integration|Thing 5 |

## Why Backups?
Well...duh
## Why dbatools?
* Automation is survival
* Scripts = Communication
  * You can't put mouse clicks in source control

## Installation & Maintenance





In [0]:
Install-Module -Name dbatools -Scope CurrentUser;
Update-Module -Name dbatools;

## How Do You Like to Back Up?
* Native or 3rd-Party?
* Ola Hallengren's Maintenance Solution?
* Custom Jobs?
* Ad-hoc?

In [0]:
Install-DbaMaintenanceSolution -SqlInstance localhost\sql17 -Solution
Backup-DbaDatabase -SqlInstance localhost\sql17 -Type Full